> # **TensorFlow  2.1.0** - ***05*** - *Aprendizagem por Reforço*



In [0]:
!pip install pandas-datareader

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque
tf.__version__

     |████████████████████████████████| 332.1MB 46kB/s 
     |████████████████████████████████| 3.0MB 49.3MB/s 
     |████████████████████████████████| 419kB 60.8MB/s 


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

'2.0.0-alpha0'

In [0]:
# Classe IA Trader para negociação no mercado de ações

class AI_Trader():
  def __init__(self, state_size, action_space= 3, model_name= 'AITrader'):
    # Quantos estados existem no ambiente (Entrada)
    self.state_size = state_size 
    # Quantidade de ações possiveis
    self.action_space = action_space
    # Experiência de Replay
    self.memory = deque(maxlen= 2000)
    self.model_name = model_name

    # Fator de desconto da equação de Bellman
    self.gamma = 0.95
    # Indica se as ações sao tomadas randomicamente ou pela rede neural (inicialmete as ações são aleatorias)
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.mode_builder()
  
  def mode_builder(self):
    # Rede Neural
    model = tf.keras.models.Sequential()
    # Camada de entrada e camanda oculta 1
    model.add(tf.keras.layers.Dense(units= 32,  activation= 'relu', input_dim= self.state_size))
    model.add(tf.keras.layers.Dense(units= 64,  activation= 'relu')) # Camada oculta 2
    model.add(tf.keras.layers.Dense(units= 128, activation= 'relu')) # Camada oculta 3
    # Camada de saida
    model.add(tf.keras.layers.Dense(units= self.action_space, activation= 'linear'))

    model.compile(loss= 'mse', optimizer= tf.keras.optimizers.Adam(lr = 0.001))
    return model
  
  def trade(self, state):
    if(random.random() <= self.epsilon):
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)): # Os ultimos registros da memória
      batch.append(self.memory[i])
    
    for state, action, reward, next_state, done in batch:
      if not done:
        # Equação de Bellman
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target, epochs= 1, verbose= 0) # verbose = 0 -> sem log de treinamento
    
    if(self.epsilon > self.epsilon_final):
      self.epsilon *= self.epsilon_decay

In [0]:
# função sigmoid
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# função para formatação dos preços
def stocks_price_format(n):
  if (n < 0):
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

print(sigmoid(0.5))
print(sigmoid(-0.5))
print(stocks_price_format(100))
print(stocks_price_format(-100))

0.6224593312018546
0.3775406687981454
$ 100.000000
- $ 100.000000


In [0]:
# pré-processamento da base de dados
dataset = data_reader.DataReader('AAPL', data_source= 'yahoo')
print(str(dataset.index[0]).split()[0])
print(dataset.shape)
dataset.head()

2010-01-04
(2557, 6)


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,30.642857,30.340000,30.490000,30.572857,123432400.0,26.538483
2010-01-05,30.798571,30.464285,30.657143,30.625713,150476200.0,26.584366
2010-01-06,30.747143,30.107143,30.625713,30.138571,138040000.0,26.161509
2010-01-07,30.285715,29.864286,30.250000,30.082857,119282800.0,26.113146
2010-01-08,30.285715,29.865715,30.042856,30.282858,111902700.0,26.286753


In [0]:
dataset['Close']

Date
2010-01-04     30.572857
2010-01-05     30.625713
2010-01-06     30.138571
2010-01-07     30.082857
2010-01-08     30.282858
                 ...    
2020-02-25    288.079987
2020-02-26    292.649994
2020-02-27    273.519989
2020-02-28    273.359985
2020-03-02    283.920013
Name: Close, Length: 2557, dtype: float64

In [0]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source= 'yahoo')
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

In [0]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
  return np.array([state]), windowed_data

In [0]:
# carregando base de dados
stock_name = 'AAPL'
data = dataset_loader(stock_name)
data.head()

Date
2010-01-04    30.572857
2010-01-05    30.625713
2010-01-06    30.138571
2010-01-07    30.082857
2010-01-08    30.282858
Name: Close, dtype: float64

In [14]:
# hyper parametros
window_size = 10
episodes = 1
batch_size = 32
data_samples = len(data)-1
# definição do modelo
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [15]:
# treinamento
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1: # Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    state = next_state
    
    if done:
      print("#" * 20)
      print(" - Total profit: {}".format(total_profit))
      print("#" * 20)
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
     
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))



  0%|          | 0/2556 [00:00<?, ?it/s]

Episode: 1/1
AI Trader bought:  $ 30.282858
AI Trader bought:  $ 30.015715
AI Trader bought:  $ 29.674286
AI Trader bought:  $ 30.092857
AI Trader bought:  $ 29.418571
AI Trader sold:  $ 30.719999  Profit: $ 0.437141
AI Trader sold:  $ 29.724285  Profit: - $ 0.291430
AI Trader bought:  $ 28.250000
AI Trader bought:  $ 29.420000
AI Trader bought:  $ 27.818571
AI Trader sold:  $ 28.461428  Profit: - $ 1.212858
AI Trader sold:  $ 27.435715  Profit: - $ 2.657143
AI Trader sold:  $ 27.922857  Profit: - $ 1.495714
AI Trader sold:  $ 27.731428  Profit: - $ 0.518572
AI Trader bought:  $ 28.027143
AI Trader sold:  $ 27.874287  Profit: - $ 1.545713
AI Trader sold:  $ 28.381428  Profit: $ 0.562857
AI Trader bought:  $ 28.625713
AI Trader bought:  $ 29.057142
AI Trader sold:  $ 28.990000  Profit: $ 0.962856
AI Trader bought:  $ 28.809999




  1%|▏         | 33/2556 [00:00<00:52, 48.13it/s]

  1%|▏         | 35/2556 [00:01<03:00, 13.99it/s]

AI Trader bought:  $ 28.665714
AI Trader sold:  $ 28.857143  Profit: $ 0.231430




  1%|▏         | 37/2556 [00:01<04:27,  9.43it/s]

...



100%|█████████▉| 2553/2556 [08:25<00:00,  4.76it/s]

100%|█████████▉| 2554/2556 [08:25<00:00,  4.92it/s]

AI Trader bought:  $ 292.649994
AI Trader sold:  $ 273.519989  Profit: - $ 24.660004




100%|█████████▉| 2555/2556 [08:25<00:00,  4.89it/s]

AI Trader bought:  $ 273.359985
####################
 - Total profit: 497.25556564331055
####################




100%|██████████| 2556/2556 [08:26<00:00,  4.85it/s]

